# Week04 data creation

Make directories and change to the directory:

    mkdir -p ~/genomes/mm10/gencode/m8
    cd ~/genomes/mm10/gencode/m8

Obtain the data:

    wget ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M8/gencode.vM8.basic.annotation.gtf.gz
    wget ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M8/gencode.vM8.pc_transcripts.fa.gz
    wget ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M8/GRCm38.primary_assembly.genome.fa.gz

## Data for STAR and `featureCounts`

Get just chr11 from the basic annotation:

    grep chr11 gencode.vM8.basic.annotation.gtf > gencode.vM8.basic.annotation.chr11.gtf

Get just chr11 from the genome fasta file:

    samtools faidx GRCm38.primary_assembly.genome.fa 
    samtools faidx GRCm38.primary_assembly.genome.fa chr11 > GRCm38.primary_assembly.chr11.fa

### STAR: Generate genome

#### Chr11 subset

Have to make the genome directory first. Use `--runDirPerm ALL_RWX` so created directory can be read by all.

    mkdir star_chr11 
    STAR \
        --runMode genomeGenerate \
        --runThreadN 4 \
        --runDirPerm All_RWX \
        --genomeFastaFiles GRCm38.primary_assembly.chr11.fa \
        --sjdbGTFfile gencode.vM8.basic.annotation.chr11.gtf \
        --genomeDir star_chr11
    
Output:

    Jan 25 12:30:21 ..... Started STAR run
    Jan 25 12:30:21 ... Starting to generate Genome files
    Jan 25 12:30:24 ... starting to sort  Suffix Array. This may take a long time...
    Jan 25 12:30:25 ... sorting Suffix Array chunks and saving them to disk...
    Jan 25 12:31:34 ... loading chunks from disk, packing SA...
    Jan 25 12:31:37 ... Finished generating suffix array
    Jan 25 12:31:37 ... Generating Suffix Array index
    Jan 25 12:32:19 ... Completed Suffix Array index
    Jan 25 12:32:19 ..... Processing annotations GTF
    Jan 25 12:32:20 ..... Inserting junctions into the genome indices
    Jan 25 12:32:39 ... writing Genome to disk ...
    Jan 25 12:32:41 ... writing Suffix Array to disk ...
    Jan 25 12:32:49 ... writing SAindex to disk
    Jan 25 12:33:03 ..... Finished successfully
    
#### Whole genome

    mkdir star
    STAR --runMode genomeGenerate --runThreadN 4 --runDirPerm All_RWX --genomeFastaFiles  GRCm38.primary_assembly.genome.fa --sjdbGTFfile gencode.vM8.basic.annotation.gtf --genomeDir star

Output:

    Jan 25 12:35:22 ..... Started STAR run
    Jan 25 12:35:22 ... Starting to generate Genome files
    Jan 25 12:36:32 ... starting to sort  Suffix Array. This may take a long time...
    Jan 25 12:36:55 ... sorting Suffix Array chunks and saving them to disk...
    Jan 25 13:10:36 ... loading chunks from disk, packing SA...
    Jan 25 13:16:30 ... Finished generating suffix array
    Jan 25 13:16:30 ... Generating Suffix Array index
    Jan 25 13:20:25 ... Completed Suffix Array index
    Jan 25 13:20:25 ..... Processing annotations GTF
    Jan 25 13:20:36 ..... Inserting junctions into the genome indices
    Jan 25 13:24:31 ... writing Genome to disk ...
    Jan 25 13:25:01 ... writing Suffix Array to disk ...
    Jan 25 13:29:56 ... writing SAindex to disk
    Jan 25 13:30:13 ..... Finished successfully


## Data for `kallisto`

Get the gene IDs from chr11:

    cut -f 9 gencode.vM8.basic.annotation.chr11.gtf  | awk -F'; ' '{print $1}' | awk -F'"' '{print $2}' | sort | uniq > gencode.vM8.chr11.gene_ids.txt


Use this list of IDs to extract 

### Create `kallisto index`

Command:

    kallisto index --index gencode.vM8.pc_transcripts.kallisto gencode.vM8.pc_transcripts.fa 

Output: 

    [build] loading fasta file gencode.vM8.pc_transcripts.fa
    [build] k-mer length: 31
    [build] warning: clipped off poly-A tail (longer than 10)
            from 274 target sequences
    [build] warning: replaced 1 non-ACGUT characters in the input sequence
            with pseudorandom nucleotides
    [build] counting k-mers ... done.
    [build] building target de Bruijn graph ...  done 
    [build] creating equivalence classes ...  done
    [build] target de Bruijn graph has 285795 contigs and contains 67545152 k-mers 


## Fastq subset

